In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries
from pathlib import Path
import os
import pickle
from functools import partial

# Save Results

## Load mlflow runs

In [2]:
results_dir = Path.cwd().parent / "results" / "real"
os.makedirs(results_dir, exist_ok=True)

In [4]:
db_port = 5101
db_name = "cohirf"
url = f"postgresql://beluccib@localhost:{db_port}/{db_name}"
# url = f"postgresql://beluccib@clust5:{db_port}/{db_name}"
engine = create_engine(url)
query = "SELECT experiments.name from experiments"
experiment_names = pd.read_sql(query, engine)["name"].tolist()

In [5]:
experiments_names = [exp for exp in experiment_names if (exp.startswith("real-"))]

In [6]:
experiments_names

['real-adjusted_rand-KernelRBFKMeans',
 'real-adjusted_rand-BatchCoHiRF-KernelRBF-1iter',
 'real-adjusted_rand-KMeans',
 'real-adjusted_rand-DBSCAN',
 'real-adjusted_rand-CoHiRF-DBSCAN',
 'real-adjusted_rand-BatchCoHiRF',
 'real-adjusted_rand-CoHiRF',
 'real-adjusted_rand-CoHiRF-KernelRBF',
 'real-adjusted_rand-BatchCoHiRF-DBSCAN-1iter',
 'real-davies_bouldin_score-BatchCoHiRF-1iter',
 'real-adjusted_mutual_info-BatchCoHiRF-KernelRBF-1iter',
 'real-silhouette-BatchCoHiRF-1iter',
 'real-silhouette-DBSCAN',
 'real-calinski_harabasz_score-BatchCoHiRF-1iter',
 'real-davies_bouldin_score-KernelRBFKMeans',
 'real-silhouette-KMeans',
 'real-davies_bouldin_score-KMeans',
 'real-calinski_harabasz_score-CoHiRF',
 'real-adjusted_mutual_info-BatchCoHiRF-DBSCAN-1iter',
 'real-davies_bouldin_score-BatchCoHiRF-KernelRBF-1iter',
 'real-adjusted_mutual_info-CoHiRF',
 'real-adjusted_mutual_info-CoHiRF-KernelRBF',
 'real-davies_bouldin_score-CoHiRF-DBSCAN',
 'real-adjusted_mutual_info-CoHiRF-DBSCAN',
 'r

In [7]:
query = "SELECT DISTINCT(key) FROM params WHERE key LIKE 'best/%%'"
best_params = pd.read_sql(query, engine)["key"].tolist()

In [8]:
params_columns = [
    "model",
    "dataset_id",
	"n_trials",
	"dataset_name",
	"standardize",
	"hpo_metric",
	"direction",
	"hpo_seed",
] + best_params

In [9]:
latest_metrics_columns = [
    "fit_model_return_elapsed_time",
    "max_memory_used_after_fit",
    "max_memory_used",
	"best/n_clusters_",
    "best/rand_score",
    "best/adjusted_rand",
    "best/mutual_info",
    "best/adjusted_mutual_info",
    "best/normalized_mutual_info",
    "best/homogeneity_completeness_v_measure",
    "best/silhouette",
    "best/calinski_harabasz_score",
    "best/davies_bouldin_score",
    "best/inertia_score",
    "best/homogeneity",
    "best/completeness",
    "best/v_measure",
    "best/elapsed_time",
]

In [10]:
tags_columns = ["raised_exception", "EXCEPTION", "mlflow.parentRunId", "Last step finished"]

In [11]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns

In [12]:
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiments_names, other_table=other_table, other_table_keys=other_table_keys)
df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [13]:
dataset_characteristics = pd.read_csv(results_dir / "datasets_characteristics.csv", index_col=0)
dataset_characteristics.index = dataset_characteristics["openml_id"].astype(str)

In [14]:
df_runs_raw_1 = df_params.join(df_latest_metrics)
df_runs_raw_1 = df_runs_raw_1.join(df_tags)

In [15]:
db_port = 5001
db_name = "cohirf"
url = f"postgresql://belucci@localhost:{db_port}/{db_name}"
# url = f"postgresql://beluccib@clust5:{db_port}/{db_name}"
engine = create_engine(url)
query = "SELECT experiments.name from experiments"
experiment_names = pd.read_sql(query, engine)["name"].tolist()
experiments_names = [exp for exp in experiment_names if (exp.startswith('real-'))]

In [16]:
experiments_names

['real-ari-BatchCoHiRF-1iter',
 'real-ari-AverageAgglomerativeClustering',
 'real-ari-BatchCoHiRF-DBSCAN-1iter',
 'real-ari-AffinityPropagation',
 'real-ari-BatchCoHiRF-SC-SRGF',
 'real-ari-CoHiRF-DBSCAN',
 'real-ari-CoHiRF-KernelRBF',
 'real-ari-CoHiRF',
 'real-ari-CompleteAgglomerativeClustering',
 'real-ari-DBSCAN',
 'real-ari-HDBSCAN',
 'real-ari-IRFLLRR',
 'real-ari-KMeans',
 'real-ari-MeanShift',
 'real-ari-OPTICS',
 'real-ari-Proclus',
 'real-ari-SingleAgglomerativeClustering',
 'real-ari-SpectralClustering',
 'real-ari-SpectralSubspaceRandomization',
 'real-ari-WardAgglomerativeClustering',
 'real-adjusted_mutual_info-BatchCoHiRF-DBSCAN-1iter',
 'real-adjusted_mutual_info-DBSCAN',
 'real-adjusted_mutual_info-BatchCoHiRF-SC-SRGF',
 'real-adjusted_mutual_info-CompleteAgglomerativeClustering',
 'real-adjusted_mutual_info-AverageAgglomerativeClustering',
 'real-adjusted_mutual_info-BatchCoHiRF-1iter',
 'real-adjusted_mutual_info-CoHiRF',
 'real-adjusted_mutual_info-CoHiRF-KernelRBF

In [17]:
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiments_names, other_table=other_table, other_table_keys=other_table_keys)
df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [18]:
df_runs_raw_2 = df_params.join(df_latest_metrics)
df_runs_raw_2 = df_runs_raw_2.join(df_tags)

In [19]:
df_runs_raw_2

key,status,start_time,end_time,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/base_model_kwargs/n_similarities,best/base_model_kwargs/sampling_ratio,best/base_model_kwargs/sc_n_clusters,best/child_run_id,...,best/rand_score,best/silhouette,best/v_measure,fit_model_return_elapsed_time,max_memory_used,max_memory_used_after_fit,EXCEPTION,Last step finished,mlflow.parentRunId,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0000158194df4bbc83acad810a6126e8,FINISHED,1753568824186,1.753569e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.220166,499.724,499.724,NaN,NaN,35bc54052da54c98b5c8f43b71499982,False
00028713c8e04978932ece53e8c94e09,FINISHED,1751922488822,1.751923e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.007838,434.324,434.324,NaN,NaN,5137e2db30a34de0bcb91ab235fccc78,False
0002f1471a6f4a639bcfd8a3d7c90f1f,FAILED,1752675967607,1.752676e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,21.065334,1492.632,1492.632,Expected a 2-dimensional container but got <cl...,NaN,4b3d5799ec4448719f0a0c5c291eda0c,True
0002f564f6e3405d859abc0abd6c448e,FAILED,1752593974468,1.752594e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.013465,455.484,455.484,could not convert string to float: 'OG',NaN,742cb3f363ef47a79da84cc6e4086844,True
000303ea6e564058a7def8cddeeda1b0,FAILED,1752585530524,1.752586e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.025954,595.816,595.816,"could not convert string to float: '""SCC""'",NaN,67862e364b9e4bf9a3a723cf9fae0873,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffd925b0d5343ec8ed94789c9587fae,FINISHED,1753589038574,1.753589e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.137964,449.936,449.936,NaN,NaN,b879ef07b7274308be2412b2655075e1,False
fffe9e46b22949c48aa2eaffc64b7028,FAILED,1752689067271,1.752689e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.067094,485.088,485.088,"could not convert string to float: '""ML2""'",NaN,8c5ee25a49bc4dd4a8c0f5960653aba4,True
fffee9233db84b49b5fa6003f8f89366,FINISHED,1753549823592,1.753550e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.073361,480.604,480.604,NaN,NaN,c462b7097d5e4812bc69f5eb17bf69f7,False


In [31]:
df_runs_raw_2["start_time"] = pd.to_datetime(df_runs_raw_2["start_time"], unit='ms')
df_runs_raw_2 = df_runs_raw_2.loc[df_runs_raw_2["start_time"] > "2025-11-01"] # to filter out old runs

In [32]:
df_runs_raw_2

key,status,start_time,end_time,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/base_model_kwargs/n_similarities,best/base_model_kwargs/sampling_ratio,best/base_model_kwargs/sc_n_clusters,best/child_run_id,...,best/rand_score,best/silhouette,best/v_measure,fit_model_return_elapsed_time,max_memory_used,max_memory_used_after_fit,EXCEPTION,Last step finished,mlflow.parentRunId,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
00076ddeca7b47d98725ca5d9b55540b,FINISHED,2025-11-06 19:17:27.877,1.762457e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.476819,448.860,448.860,NaN,_on_train_end,7efea00adc674d8784479417e9e429f0,False
00076f100a1b45c48445040322e4623d,FINISHED,2025-11-06 19:51:49.007,1.762459e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.480747,602.064,602.064,NaN,_on_train_end,553d6499305a48bfadfe5c78651cca55,False
000c233cf5ac453db6785ebb677e64c2,FINISHED,2025-11-06 18:40:30.894,1.762455e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.535595,467.380,467.380,NaN,_on_train_end,d89d4f7095c64f9db665822c8597f06c,False
000cdafad08c47e78d0a424e8e135071,FINISHED,2025-11-06 19:22:42.851,1.762457e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.547577,379.724,379.724,NaN,_on_train_end,e315e659a4b045d29de7aa2459943764,False
000fa0cf9759482ba8b3edec3e133db7,FINISHED,2025-11-06 19:07:15.184,1.762456e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.494779,467.056,467.056,NaN,_on_train_end,ea2ee81c7795407b8fa8cfb164327cd9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff657133e5948e1b4a3deb21a8b2a3f,FINISHED,2025-11-06 19:04:34.537,1.762456e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.710222,464.168,464.168,NaN,_on_train_end,0019c00fbd4243f79f5c61c50650227f,False
fff6a206756f4e0db1836b9c1d7b3ce0,FINISHED,2025-11-06 20:58:54.147,1.762463e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.179803,415.036,415.036,NaN,_on_train_end,f3d4d5af78864b27a73b0091d2791fb9,False
fff74c1d2e7f48e18880666740b7b23d,FINISHED,2025-11-06 19:48:04.373,1.762459e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.059326,372.436,372.436,NaN,_on_train_end,b9fdde667dc94d82a939ca8512da2f2b,False


In [34]:
df_runs_raw = pd.concat([df_runs_raw_1, df_runs_raw_2], axis=0)
df_runs_raw = df_runs_raw.join(dataset_characteristics, on="dataset_id", rsuffix="_dataset")
df_runs_raw.to_csv(results_dir / 'df_runs_raw_cer_tgcc.csv', index=True)

In [35]:
df_runs_raw = pd.read_csv(results_dir / "df_runs_raw_cer_tgcc.csv", index_col=0, low_memory=False)
df_runs_raw["model"] = df_runs_raw["model"] + "-" + df_runs_raw["n_trials"].astype(int).astype(str)
df_runs_raw_parents = df_runs_raw.copy()
df_runs_raw_parents = df_runs_raw_parents.loc[df_runs_raw_parents["mlflow.parentRunId"].isna()]

In [36]:
df_runs_raw_parents.head(5)

,status,start_time,end_time,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/base_model_kwargs/n_similarities,best/base_model_kwargs/sampling_ratio,best/base_model_kwargs/sc_n_clusters,best/child_run_id,...,EXCEPTION,Last step finished,mlflow.parentRunId,raised_exception,dataset,openml_id,n_instances,n_features,n_classes,n_categorical
run_uuid,,,,,,,,,,,,,,,,,,,,,
0001f124c40c40b6b902fcabc76e7eea,FINISHED,1757511120768,1.757511e+12,NaN,NaN,NaN,NaN,NaN,NaN,c854172d93d945b5b98d1d2e7f8b8244,...,NaN,NaN,NaN,False,iris,61,150,5,3,1
00027aa26863479689cb9159eb05d27d,FINISHED,1759251752094,1.759254e+12,NaN,NaN,21.0,NaN,NaN,NaN,c968bd4c602b4cc1b80ea7ebf3ef514a,...,NaN,NaN,NaN,False,mnist_784,554,70000,785,10,1
000ed0a4c2254d48a2ea293ecf9480b9,FINISHED,1761159995601,1.761160e+12,NaN,NaN,NaN,NaN,NaN,NaN,775080a847654e7885ed3003bbb9d622,...,NaN,_on_train_end,NaN,False,nursery,1568,12958,9,4,9
0016b21b09e44c7595b26130cbf448f9,FINISHED,1757524956961,1.757525e+12,NaN,NaN,3.0,NaN,NaN,NaN,318c5fee63ac48ee93c0f8897a15e567,...,NaN,NaN,NaN,False,golub-1999-v2,46780,72,1869,3,0
001b3f77c3e44c12b14761e15c808ec9,FINISHED,1757510889568,1.757511e+12,NaN,NaN,NaN,NaN,NaN,NaN,c3df07fdb54f487a922cb765d3b9c0a7,...,NaN,NaN,NaN,False,ecoli,39,336,8,8,1


## Delete duplicate runs (if any) and complete some models that cannot run with some datasets

In [37]:
non_duplicate_columns = [
    "model",
    "dataset_id",
	"standardize",
	"hpo_metric",
	"hpo_seed",
]
# df_runs_parents.loc[df_runs_parents["best/n_clusters_"]*0.5 > df_runs_parents["n_instances"], "best/adjusted_rand"] = 
df_runs_parents = df_runs_raw_parents.dropna(axis=0, how="all", subset=["best/adjusted_rand"]).copy()
# add back runs that were not evaluated because we judged too many clusters (but they run anyway)
# df_valid_runs = df_runs_raw_parents.loc[df_runs_raw_parents["best/n_clusters_"] > df_runs_raw_parents["n_instances"]*0.5].copy()
# df_runs_parents = pd.concat([df_runs_parents, df_valid_runs], axis=0)
df_runs_parents = df_runs_parents.loc[(~df_runs_parents.duplicated(non_duplicate_columns))]
# fill missing values with "None"
df_runs_parents = df_runs_parents.fillna("None")

In [38]:
# get number of children runs that raised exception for each parent run
children_exceptions = df_runs_raw.groupby("mlflow.parentRunId")["raised_exception"].sum()
df_runs_parents["n_children_raised_exception"] = df_runs_parents.index.map(children_exceptions).fillna(0)

In [39]:
df_runs_parents.loc[(df_runs_parents["n_children_raised_exception"] > 0) & (df_runs_parents["raised_exception"] == False) & (df_runs_parents["model"].str.find("SC-SRGF") == -1), ["dataset_id", "model", "hpo_metric", "n_children_raised_exception"]]

,dataset_id,model,hpo_metric,n_children_raised_exception
run_uuid,,,,
0523d99695664a3294fa128a1eae79af,554,CoHiRF-DBSCAN-60,calinski_harabasz_score,9
075cbb1e8b9f41a0b1b1cb54ff6673af,39,CoHiRF-top-down-60,davies_bouldin_score,4
08433992474241cdbe44aca2692cd7df,554,CoHiRF-DBSCAN-60,davies_bouldin_score,2
086989a27cdb467b85ed2248ce8b32d3,554,CoHiRF-DBSCAN-60,calinski_harabasz_score,8
0abf67098276481cabbcb002ae6bdf84,1568,CoHiRF-top-down-60,calinski_harabasz_score,2
...,...,...,...,...
f7504ad3ccad429e82fb12541282747b,40685,CoHiRF-top-down-60,calinski_harabasz_score,1
f84f03dfb6964911a05142a3072c699f,1568,CoHiRF-top-down-60,adjusted_mutual_info,3
fa554932b2f842d6833bc8331388e3f7,47039,BatchCoHiRF-1iter-60,adjusted_rand,7


In [40]:
df_to_cat = []
hpo_metrics = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    "silhouette",
    "davies_bouldin_score",
    "normalized_mutual_info",
]
standardize = [True]
hpo_seed = [i for i in range(5)]
fill_value = pd.NA
fill_columns = ["best/adjusted_rand", "best/adjusted_mutual_info", "best/calinski_harabasz_score", "best/silhouette", "best/davies_bouldin_score", "best/normalized_mutual_info"]

In [51]:
# Too memory intensive
dataset_ids_to_complete = [182, 554, 1478, 1568, 40685]
model_names = [
    "CoHiRF-SC-SRGF-60",
    "CoHiRF-SC-SRGF-top-down-60",
    "CoHiRF-SC-SRGF-top-down-inv-60",
    "SpectralSubspaceRandomization-60",
    "CoHiRF-SC-SRGF-1R-60",
    "CoHiRF-SC-SRGF-top-down-1R-60",
    "CoHiRF-SC-SRGF-top-down-inv-1R-60",
    "CoHiRF-SC-SRGF-2R-60",
    "CoHiRF-SC-SRGF-top-down-2R-60",
	"CoHiRF-SC-SRGF-top-down-inv-2R-60",
]
for dataset_id in dataset_ids_to_complete:
    for model_name in model_names:
        for hpo_metric in hpo_metrics:
            for std in standardize:
                for seed in hpo_seed:
                    new_row = {
						"dataset_id": dataset_id,
						"model": model_name,
						"hpo_metric": hpo_metric,
						"standardize": std,
						"hpo_seed": seed
					}
                    for col in fill_columns:
                        new_row[col] = fill_value
                    df_to_cat.append(new_row)

In [52]:
# # Too few examples (<100) to run in batch
# dataset_ids_to_complete = [46773, 46774, 46775, 46776, 46777, 46779, 46780, 46781]
# model_names = [
#     "BatchCoHiRF-1iter-30",
#     "BatchCoHiRF-DBSCAN-1iter-30",
#     "BatchCoHiRF-SC-SRGF-30",
#     "BatchCoHiRF-KernelRBF-1iter-30",
#     "BatchCoHiRF-1iter-60",
# 	"BatchCoHiRF-DBSCAN-1iter-60",
# 	# "BatchCoHiRF-SC-SRGF-60",
# 	# "BatchCoHiRF-KernelRBF-1iter-60",
# ]
# for dataset_id in dataset_ids_to_complete:
# 	for model_name in model_names:
# 		for hpo_metric in hpo_metrics:
# 			for std in standardize:
# 				mask = (df_runs_parents["dataset_id"] == dataset_id) & (df_runs_parents["model"] == model_name) & (df_runs_parents["hpo_metric"] == hpo_metric) & (df_runs_parents["standardize"] == std)
# 				if not mask.any():
# 					new_row = {
# 						"dataset_id": dataset_id,
# 						"model": model_name,
# 						"hpo_metric": hpo_metric,
# 						"standardize": std,
# 					}
# 					for col in fill_columns:
# 						new_row[col] = fill_value
# 					df_to_cat.append(new_row)

In [53]:
# # Too many examples for IRFLLRR
# dataset_ids_to_complete = [40685]
# model_names = [
#     "IRFLLRR-30",
#     "IRFLLRR-60",
# ]
# hpo_metrics = ["adjusted_rand", "adjusted_mutual_info", "calinski_harabasz_score", "silhouette", "davies_bouldin_score", "normalized_mutual_info"]
# standardize = [True]
# fill_value = pd.NA
# fill_columns = ["best/adjusted_rand", "best/adjusted_mutual_info", "best/calinski_harabasz_score", "best/silhouette", "best/davies_bouldin_score", "best/normalized_mutual_info"]
# for dataset_id in dataset_ids_to_complete:
#     for model_name in model_names:
#         for hpo_metric in hpo_metrics:
#             for std in standardize:
#                 mask = (
#                     (df_runs_parents["dataset_id"] == dataset_id)
#                     & (df_runs_parents["model"] == model_name)
#                     & (df_runs_parents["hpo_metric"] == hpo_metric)
#                     & (df_runs_parents["standardize"] == std)
#                 )
#                 if not mask.any():
#                     new_row = {
#                         "dataset_id": dataset_id,
#                         "model": model_name,
#                         "hpo_metric": hpo_metric,
#                         "standardize": std,
#                     }
#                     for col in fill_columns:
#                         new_row[col] = fill_value
#                     df_to_cat.append(new_row)

In [54]:
df_runs_parents = pd.concat([df_runs_parents, pd.DataFrame(df_to_cat)], axis=0)

# Missing

In [55]:
model_nickname = df_runs_parents['model'].unique().tolist()
model_nickname.sort()
model_nickname

['BatchCoHiRF-1iter-60',
 'BatchCoHiRF-1iter-random-60',
 'BatchCoHiRF-1iter-stratified-60',
 'BatchCoHiRF-60',
 'BatchCoHiRF-DBSCAN-1iter-60',
 'BatchCoHiRF-DBSCAN-1iter-random-60',
 'BatchCoHiRF-DBSCAN-1iter-stratified-60',
 'BatchCoHiRF-KernelRBF-1iter-60',
 'BatchCoHiRF-KernelRBF-1iter-random-60',
 'BatchCoHiRF-KernelRBF-1iter-stratified-60',
 'BatchCoHiRF-SC-SRGF-1iter-60',
 'BatchCoHiRF-SC-SRGF-2-60',
 'CoHiRF-1000-60',
 'CoHiRF-60',
 'CoHiRF-DBSCAN-60',
 'CoHiRF-KernelRBF-60',
 'CoHiRF-SC-SRGF-1R-60',
 'CoHiRF-SC-SRGF-2R-60',
 'CoHiRF-SC-SRGF-60',
 'CoHiRF-SC-SRGF-top-down-1R-60',
 'CoHiRF-SC-SRGF-top-down-2R-60',
 'CoHiRF-SC-SRGF-top-down-60',
 'CoHiRF-SC-SRGF-top-down-inv-1R-60',
 'CoHiRF-SC-SRGF-top-down-inv-2R-60',
 'CoHiRF-SC-SRGF-top-down-inv-60',
 'CoHiRF-top-down-60',
 'CoHiRF-top-down-inv-60',
 'DBSCAN-60',
 'KMeans-60',
 'KernelRBFKMeans-60',
 'SpectralSubspaceRandomization-60']

In [56]:
non_duplicate_columns = [
	"model",
	"dataset_id",
	"standardize",
	"hpo_metric",
	"hpo_seed",
]

In [64]:
model_nickname = [
    "BatchCoHiRF-1iter-60",
    # "BatchCoHiRF-1iter-random-60",
    # "BatchCoHiRF-1iter-stratified-60",
    "BatchCoHiRF-DBSCAN-1iter-60",
    # "BatchCoHiRF-DBSCAN-1iter-random-60",
    # "BatchCoHiRF-DBSCAN-1iter-stratified-60",
    "BatchCoHiRF-KernelRBF-1iter-60",
    # "BatchCoHiRF-KernelRBF-1iter-random-60",
    # "BatchCoHiRF-KernelRBF-1iter-stratified-60",
    "BatchCoHiRF-SC-SRGF-1iter-60",
    # "BatchCoHiRF-SC-SRGF-1iter-random-60",
    # "BatchCoHiRF-SC-SRGF-1iter-stratified-60",
    # "BatchCoHiRF-SC-SRGF-2-60",
    "CoHiRF-60",
    "CoHiRF-top-down-60",
    "CoHiRF-top-down-inv-60",
    # "CoHiRF-1000-60",
    "CoHiRF-DBSCAN-60",
    "CoHiRF-DBSCAN-top-down-60",
    "CoHiRF-DBSCAN-top-down-inv-60",
    "CoHiRF-KernelRBF-60",
	"CoHiRF-KernelRBF-top-down-60",
    "CoHiRF-KernelRBF-top-down-inv-60",
    "CoHiRF-SC-SRGF-60",
    "CoHiRF-SC-SRGF-1R-60",
    "CoHiRF-SC-SRGF-1R-top-down-60",
    "CoHiRF-SC-SRGF-1R-top-down-inv-60",
    "DBSCAN-60",
    "KMeans-60",
    "KernelRBFKMeans-60",
    "SpectralSubspaceRandomization-60",
]
dataset_id = [
    39,
    61,
    182,
    1478,
    1568,
    40685,
    40984,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
    46782,
    46783,
    554,
    # 1110,
    # 47039
]
standardize = [True]
hpo_metric = [
    "adjusted_rand",
    # "adjusted_mutual_info",
    "calinski_harabasz_score",
    # "normalized_mutual_info",
    # "davies_bouldin_score",
    "silhouette",
]
hpo_seed = [i for i in range(5)]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric, hpo_seed]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)
df_missing

,model,dataset_id,standardize,hpo_metric,hpo_seed
0,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,0
1,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,1
2,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,2
3,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,3
4,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,4
...,...,...,...,...,...
1750,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,0
1751,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,1
1752,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,2
1753,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,3


In [47]:
model_nickname = [
    "BatchCoHiRF-1iter-60",
    # "BatchCoHiRF-1iter-random-60",
    # "BatchCoHiRF-1iter-stratified-60",
    "BatchCoHiRF-DBSCAN-1iter-60",
    # "BatchCoHiRF-DBSCAN-1iter-random-60",
    # "BatchCoHiRF-DBSCAN-1iter-stratified-60",
    "BatchCoHiRF-KernelRBF-1iter-60",
    # "BatchCoHiRF-KernelRBF-1iter-random-60",
    # "BatchCoHiRF-KernelRBF-1iter-stratified-60",
    "BatchCoHiRF-SC-SRGF-1iter-60",
    # "BatchCoHiRF-SC-SRGF-1iter-random-60",
    # "BatchCoHiRF-SC-SRGF-1iter-stratified-60",
    # "BatchCoHiRF-SC-SRGF-2-60",
    "CoHiRF-60",
    "CoHiRF-top-down-60",
    "CoHiRF-top-down-inv-60",
    # "CoHiRF-1000-60",
    "CoHiRF-DBSCAN-60",
    "CoHiRF-KernelRBF-60",
    "CoHiRF-SC-SRGF-60",
    "DBSCAN-60",
    "KMeans-60",
    "KernelRBFKMeans-60",
    "SpectralSubspaceRandomization-60",
]
dataset_id = [
    39,
    61,
    182,
    1478,
    1568,
    40685,
    40984,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
    46782,
    46783,
    554,
    # 1110,
    # 47039
]
standardize = [True]
hpo_metric = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    # "normalized_mutual_info",
    "davies_bouldin_score",
    "silhouette",
]
hpo_seed = [i for i in range(5)]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric, hpo_seed]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)
df_missing

,model,dataset_id,standardize,hpo_metric,hpo_seed


In [48]:
model_nickname = [
    "CoHiRF-1000-60",
]
dataset_id = [
    182,
    1478,
    1568,
    40685,
    40984,
    46782,
    46783,
    554,
]
standardize = [True]
hpo_metric = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    # "normalized_mutual_info",
    "davies_bouldin_score",
    "silhouette",
]
hpo_seed = [i for i in range(5)]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric, hpo_seed]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)
df_missing

,model,dataset_id,standardize,hpo_metric,hpo_seed


In [49]:
model_nickname = [
    "BatchCoHiRF-1iter-60",
    # "BatchCoHiRF-1iter-random-60",
    # "BatchCoHiRF-1iter-stratified-60",
    "BatchCoHiRF-DBSCAN-1iter-60",
    # "BatchCoHiRF-DBSCAN-1iter-random-60",
    # "BatchCoHiRF-DBSCAN-1iter-stratified-60",
    "BatchCoHiRF-KernelRBF-1iter-60",
    # "BatchCoHiRF-KernelRBF-1iter-random-60",
    # "BatchCoHiRF-KernelRBF-1iter-stratified-60",
    "BatchCoHiRF-SC-SRGF-1iter-60",
    # "BatchCoHiRF-SC-SRGF-1iter-random-60",
    # "BatchCoHiRF-SC-SRGF-1iter-stratified-60",
    # "BatchCoHiRF-SC-SRGF-2-60",
    # "CoHiRF-60",
    # "CoHiRF-1000-60",
    # "CoHiRF-DBSCAN-60",
    # "CoHiRF-KernelRBF-60",
    # "CoHiRF-SC-SRGF-60",
    # "DBSCAN-60",
    "KMeans-60",
    # "KernelRBFKMeans-60",
    # "SpectralSubspaceRandomization-60",
]
dataset_id = [47039]
standardize = [True]
hpo_metric = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    # "normalized_mutual_info",
    # "davies_bouldin_score",
    # "silhouette",
]
hpo_seed = [i for i in range(5)]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric, hpo_seed]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)
df_missing

,model,dataset_id,standardize,hpo_metric,hpo_seed
0,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_rand,0
1,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_rand,1
2,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_rand,2
3,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_rand,3
4,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_rand,4
5,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_mutual_info,0
6,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_mutual_info,1
7,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_mutual_info,2
8,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_mutual_info,3
9,BatchCoHiRF-SC-SRGF-1iter-60,47039,True,adjusted_mutual_info,4


In [50]:
model_nickname = ["CoHiRF-SC-SRGF-1R-60", "CoHiRF-SC-SRGF-2R-60", "CoHiRF-SC-SRGF-60", "SpectralSubspaceRandomization-60"]
dataset_id = [
    39,
    61,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
]
standardize = [True]
hpo_metric = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    # "normalized_mutual_info",
    "davies_bouldin_score",
    "silhouette",
]
hpo_seed = [i for i in range(5)]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric, hpo_seed]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)
df_missing

,model,dataset_id,standardize,hpo_metric,hpo_seed
0,CoHiRF-SC-SRGF-2R-60,39,True,adjusted_rand,2


In [58]:
# Join df_runs_raw_parents into df_missing using non_duplicate_columns to get the EXCEPTION column
df_missing_with_exception = df_missing.merge(
    df_runs_raw_parents[non_duplicate_columns + ["raised_exception", "EXCEPTION", "Last step finished"]],
    how="left",
    left_on=["model", "dataset_id", "standardize", "hpo_metric", "hpo_seed"],
    right_on=["model", "dataset_id", "standardize", "hpo_metric", "hpo_seed"],
)
df_missing_with_exception[
    [
        "model",
        "dataset_id",
        "standardize",
        "hpo_metric",
        "hpo_seed",
        "raised_exception",
        "EXCEPTION",
        "Last step finished",
    ]
]

,model,dataset_id,standardize,hpo_metric,hpo_seed,raised_exception,EXCEPTION,Last step finished
0,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,0,NaN,NaN,NaN
1,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,1,NaN,NaN,NaN
2,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,2,NaN,NaN,NaN
3,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,3,NaN,NaN,NaN
4,CoHiRF-DBSCAN-top-down-60,39,True,adjusted_rand,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2920,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,0,NaN,NaN,NaN
2921,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,1,NaN,NaN,NaN
2922,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,2,NaN,NaN,NaN
2923,CoHiRF-SC-SRGF-1R-top-down-inv-60,554,True,silhouette,3,NaN,NaN,NaN


In [65]:
df_missing_dict = df_missing.copy()
# get only rows from high_mem_tuples
# df_missing_dict = df_missing_dict.merge(high_mem_tuples, on=["model", "dataset_id"], how="left", indicator=True)
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "both"].drop(columns="_merge")
# exclude rows that are in missing_ari_tuples
# df_missing_dict = df_missing_dict.merge(
# 	missing_ari_tuples, on=["model", "dataset_id"], how="left", indicator=True
# )|
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")
# exclude rows that are in high_mem_tuples
# df_missing_dict = df_missing_dict.merge(
# 	high_mem_tuples, on=["model", "dataset_id"], how="left", indicator=True
# )
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")
# to_drop = pd.concat([missing_ari_tuples, high_mem_tuples], ignore_index=True)
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")

In [68]:
# get rid of -60
df_missing_dict["model"] = df_missing_dict["model"].str.replace("-60", "")
df_missing_dict["seed_dataset_order"] = df_missing_dict["hpo_seed"]
df_missing_dict = df_missing_dict.loc[~df_missing_dict["dataset_id"].isin([46782, 46783, 1478, 1568, 40984, 40685])]
df_missing_dict.to_csv(results_dir / "df_missing_dict.csv", index=False)

In [67]:
df_missing_dict

,model,dataset_id,standardize,hpo_metric,hpo_seed,seed_dataset_order
0,CoHiRF-DBSCAN-top-down,39,True,adjusted_rand,0,0
1,CoHiRF-DBSCAN-top-down,39,True,adjusted_rand,1,1
2,CoHiRF-DBSCAN-top-down,39,True,adjusted_rand,2,2
3,CoHiRF-DBSCAN-top-down,39,True,adjusted_rand,3,3
4,CoHiRF-DBSCAN-top-down,39,True,adjusted_rand,4,4
...,...,...,...,...,...,...
1750,CoHiRF-SC-SRGF-1R-top-down-inv,554,True,silhouette,0,0
1751,CoHiRF-SC-SRGF-1R-top-down-inv,554,True,silhouette,1,1
1752,CoHiRF-SC-SRGF-1R-top-down-inv,554,True,silhouette,2,2
1753,CoHiRF-SC-SRGF-1R-top-down-inv,554,True,silhouette,3,3


# Tables

In [66]:
def get_parameters_string(row):
    parameter_names = {
		"best/alpha": "\\alpha",
		"best/avg_dims": "d",
		"best/base_model_kwargs/eps": "\\epsilon",
		"best/base_model_kwargs/min_samples": "n_{\\text{min}}",
		"best/base_model_kwargs/n_clusters": "C",
		"best/c": "c",
		"best/cohirf_kwargs/base_model_kwargs/eps": "\\epsilon",
		"best/cohirf_kwargs/base_model_kwargs/min_samples": "n_{\\text{min}}",
		"best/cohirf_kwargs/kmeans_n_clusters": "C",
		"best/cohirf_kwargs/n_features": "q",
		"best/cohirf_kwargs/repetitions": "R",
		"best/damping": "\\lambda",
		# "best/density_threshold": "\\tau",
		"best/eps": "\\epsilon",
		"best/kmeans_n_clusters": "C",
		"best/lambda_": "\\lambda",
		"best/min_bin_freq": "bin_{\\text{min}}",
		"best/min_cluster_size": "C_{\\text{min}}",
		"best/min_samples": "n_{\\text{min}}",
		"best/n_clusters": "C",
		"best/n_features": "q",
		# "best/n_partitions": "P",
		"best/n_similarities": "m",
		"best/p": "p",
		"best/repetitions": "R",
		"best/sampling_ratio": "r",
		"best/sc_n_clusters": "C",
		"best/transform_kwargs/gamma": "\\gamma",
	}
    first = True
    str = ""
    for p in parameter_names.keys():
        if not pd.isna(row[p]) and row[p] != "None":
            if not first:
                str += "; "
            else:
                first = False
            value = float(row[p])
            if value.is_integer():
                value = int(value)
                str += f"${parameter_names[p]}={value}$"
            else:
                str += f"${parameter_names[p]}={value:0.2f}$"
    return str

In [67]:
def highlight_max(df, column_name, level=0):
    df_column = df[column_name]
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_highlighted] = 'font-weight: bold'
    return df_css

In [68]:
def highlight_min(df, column_name, level=0):
    df_column = df[column_name]
    min_values = df_column.groupby(level=level).transform("min")
    is_highlighted = df_column.round(3) == min_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ""
    df_css[is_highlighted] = "font-weight: bold"
    return df_css

In [69]:
def highlight_max_index(series_index, df_column, level=0):
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css[:] = ''
    series_css[is_highlighted.values] = 'font-weight: bold'
    return series_css

In [70]:
def underline_2nd_max(df, column_name, level=0):
    df_column = df[column_name]
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.round(3).drop_duplicates().nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_underlined] = 'underline: --latex--rwrap'
    return df_css

In [71]:
def underline_2nd_min(df, column_name, level=0):
    df_column = df[column_name]
    # get the second min value
    second_min_values = df_column.groupby(level=level).transform(
        lambda x: x.round(3).drop_duplicates().nsmallest(2).iloc[-1]
    )
    is_underlined = df_column.round(3) == second_min_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ""
    df_css[is_underlined] = "underline: --latex--rwrap"
    return df_css

In [72]:
def underline_2nd_max_index(series_index, df_column, level=0):
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css.loc[:] = ''
    series_css[is_underlined.values] = 'underline: --latex--rwrap'
    return series_css

## Some Models

In [73]:
print(*df_runs_parents['model'].unique(), sep="\n")

CoHiRF-SC-SRGF-60
BatchCoHiRF-SC-SRGF-1iter-60
SpectralSubspaceRandomization-60
BatchCoHiRF-SC-SRGF-2-60
CoHiRF-SC-SRGF-2R-60
CoHiRF-SC-SRGF-1R-60


In [82]:
model_names = {
    'CoHiRF-SC-SRGF-60': 'CoHiRF-SC-SRGF',
    'SpectralSubspaceRandomization-60': "SC-SRGF",
    'CoHiRF-SC-SRGF-2R-60': 'CoHiRF-SC-SRGF-2R',
    'CoHiRF-SC-SRGF-1R-60': 'CoHiRF-SC-SRGF-1R',
}

dataset_names = {
    "binary_alpha_digits": "binary-alpha-digits",
	"mnist_784": "mnist",
}  # otherwise we get an error in latex

dataset_id = [
    39,
    61,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
]

# Filter to only standardized runs
df = df_runs_parents.copy()
df = df.loc[df['standardize'] == True]
df = df.loc[df['model'].isin(model_names.keys())]
df = df.loc[df["dataset_id"].isin(dataset_id)]
df = df.replace({"model": model_names})
df = df.replace({"dataset_name": dataset_names})

# Filter to only runs with hpo_seed in range(5)
df = df.loc[df['hpo_seed'].isin(range(5))]

In [83]:
hpo_metrics = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    "silhouette",
    "davies_bouldin_score",
    "normalized_mutual_info",
]

hpo_metrics_rename = [
    "ARI",
    "AMI",
    "Calinski",
    "Silhouette",
    "Davies-Bouldin",
    "NMI",
]

dfs_metrics = {}

for hpo_metric, hpo_metric_rename in zip(hpo_metrics, hpo_metrics_rename):
    df_metric = df.loc[df['hpo_metric'] == hpo_metric][
        ['dataset_name', 'model', 'hpo_seed', f'best/{hpo_metric}']
    ].rename(columns={f'best/{hpo_metric}': hpo_metric_rename})
    df_metric = df_metric.dropna(subset=[hpo_metric_rename])
    df_metric = df_metric.set_index(['dataset_name', 'model', 'hpo_seed'])
    df_metric = df_metric.astype({hpo_metric_rename: float})
    dfs_metrics[hpo_metric_rename] = df_metric

df_metrics = pd.concat(dfs_metrics.values(), axis=1, join="outer")
df_metrics = df_metrics.reset_index()

# calculate mean and std
df_metrics = df_metrics.groupby(['dataset_name', 'model']).agg(['mean', 'std'])
# flatten multiindex columns
df_metrics.columns = [' '.join(col).strip() for col in df_metrics.columns.values]
# drop hpo_seed level
df_metrics = df_metrics.drop(columns=['hpo_seed mean', 'hpo_seed std'])
# Rename index levels
df_metrics.index.names = ["Dataset", "Model"]
# df_metrics["Davies-Bouldin"] = df_metrics["Davies-Bouldin"].astype(float)
# create columns Metric (Mean ± Std)
# for metric in hpo_metrics_rename:
#     df_metrics[f"{metric}"] = df_metrics[f"{metric} mean"].round(3).astype(str) + " $\\pm$ " + df_metrics[f"{metric} std"].round(3).astype(str)

for metric in hpo_metrics_rename:
    df_metrics[f"{metric}"] = (
        df_metrics[f"{metric} mean"].apply(lambda x: f"{x:.3f}" if not pd.isna(x) else "No Run")
        + " $\\pm$ "
        + df_metrics[f"{metric} std"].apply(lambda x: f"{x:.3f}" if not pd.isna(x) else "No Run")
    )


# Reset Seed level
# df_metrics = df_metrics.reset_index(level="Seed")

In [84]:
df_metrics

ARI mean   ARI std  AMI mean  \
Dataset           Model                                             
alizadeh-2000-v2  CoHiRF-SC-SRGF     0.632841  0.042223  0.757751   
                  CoHiRF-SC-SRGF-1R  0.947128  0.000000  0.922018   
                  CoHiRF-SC-SRGF-2R  0.947128  0.000000  0.922018   
                  SC-SRGF            0.891422  0.124562  0.854421   
alizadeh-2000-v3  CoHiRF-SC-SRGF     0.442624  0.012813  0.627770   
                  CoHiRF-SC-SRGF-1R  0.519160  0.000000  0.723841   
                  CoHiRF-SC-SRGF-2R  0.519160  0.000000  0.723841   
                  SC-SRGF            0.519160  0.000000  0.702416   
armstrong-2002-v1 CoHiRF-SC-SRGF     0.371617  0.039012  0.444932   
                  CoHiRF-SC-SRGF-1R  0.462629  0.116528  0.402909   
                  CoHiRF-SC-SRGF-2R  0.445998  0.110654  0.419080   
                  SC-SRGF            0.313416  0.045964  0.420515   
bittner-2000      CoHiRF-SC-SRGF     0.057890  0.022848  0.103082   
                  CoHiRF-SC-SRGF-1R  0.169038  0.019309  0.223727   
                  CoHiRF-SC-SRGF-2R  0.140731  0.010381  0.190714   
                  SC-SRGF            0.131626  0.034718  0.204931   
bredel-2005       CoHiRF-SC-SRGF     0.326251  0.024664  0.413455   
                  CoHiRF-SC-SRGF-1R  0.655139  0.036821  0.524535   
                  CoHiRF-SC-SRGF-2R  0.653397  0.035275  0.530495   
                  SC-SRGF            0.639952  0.034435  0.538158   
chowdary-2006     CoHiRF-SC-SRGF     0.702278  0.041618  0.646869   
                  CoHiRF-SC-SRGF-1R  0.923766  0.000000  0.858643   
                  CoHiRF-SC-SRGF-2R  0.923766  0.000000  0.858643   
                  SC-SRGF            0.923766  0.000000  0.858643   
ecoli             CoHiRF-SC-SRGF     0.723492  0.011082  0.667921   
                  CoHiRF-SC-SRGF-1R  0.723821  0.002943  0.681983   
                  CoHiRF-SC-SRGF-2R  0.729594  0.010561  0.695410   
                  SC-SRGF            0.713038  0.007713  0.676466   
garber-2001       CoHiRF-SC-SRGF     0.437160  0.054210  0.323151   
                  CoHiRF-SC-SRGF-1R  0.235458  0.012791  0.264929   
                  CoHiRF-SC-SRGF-2R  0.454286  0.033884  0.332909   
                  SC-SRGF            0.204396  0.035002  0.251175   
golub-1999-v2     CoHiRF-SC-SRGF     0.352280  0.043072  0.451229   
                  CoHiRF-SC-SRGF-1R  0.377647  0.053731  0.466433   
                  CoHiRF-SC-SRGF-2R  0.367010  0.054796  0.452678   
                  SC-SRGF            0.298271  0.030361  0.454763   
iris              CoHiRF-SC-SRGF     0.726146  0.031791  0.717521   
                  CoHiRF-SC-SRGF-1R  0.885958  0.008285  0.860005   
                  CoHiRF-SC-SRGF-2R  0.826558  0.036383  0.803129   
                  SC-SRGF            0.776839  0.077965  0.849873   
khan-2001         CoHiRF-SC-SRGF     0.403977  0.017035  0.552097   
                  CoHiRF-SC-SRGF-1R  0.405887  0.000000  0.555187   
                  CoHiRF-SC-SRGF-2R  0.423957  0.006823  0.573815   
                  SC-SRGF            0.364275  0.038585  0.522855   

                                          AMI std  Calinski mean  \
Dataset           Model                                            
alizadeh-2000-v2  CoHiRF-SC-SRGF     3.062353e-02       7.421898   
                  CoHiRF-SC-SRGF-1R  0.000000e+00      12.338328   
                  CoHiRF-SC-SRGF-2R  0.000000e+00      12.338328   
                  SC-SRGF            9.259718e-02      12.338328   
alizadeh-2000-v3  CoHiRF-SC-SRGF     1.897289e-02       7.411181   
                  CoHiRF-SC-SRGF-1R  0.000000e+00      12.340749   
                  CoHiRF-SC-SRGF-2R  0.000000e+00      12.340749   
                  SC-SRGF            4.790779e-02      12.340749   
armstrong-2002-v1 CoHiRF-SC-SRGF     1.701540e-02       3.423828   
                  CoHiRF-SC-SRGF-1R  1.630933e-02       4.856990   
                  CoHiRF-SC-SRGF-2R  1.638856e

In [85]:
# Add mean time columns to the existing df_metrics dataframe
# Using the same filtering approach as the original df_metrics
# df = df_runs_parents.copy()
# df = df.loc[df["standardize"] == True]
# df = df.loc[df["model"].isin(model_names.keys())]
# df = df.replace({"model": model_names})
# df = df.replace({"dataset_name": dataset_names})
# # Filter to only runs with hpo_seed in range(5)
# df = df.loc[df["hpo_seed"].isin(range(5))]

# Calculate mean and std times for each dataset-model combination across all metrics
df_times = (
    df.groupby(["dataset_name", "model"])
    .agg({"best/elapsed_time": ["mean", "std"], "fit_model_return_elapsed_time": ["mean", "std"]})
    .rename(columns={"best/elapsed_time": "Best Time", "fit_model_return_elapsed_time": "HPO Time"})
)

# Flatten multiindex columns
df_times.columns = [' '.join(col).strip() for col in df_times.columns.values]
# Set the same index structure as df_metrics
df_times.index.names = ["Dataset", "Model"]

df_times["Best Time"] = (
	df_times["Best Time mean"].apply(lambda x: f"{x:4.3f}" if not pd.isna(x) else "No Run")
	+ " $\\pm$ " 
	+ df_times["Best Time std"].apply(lambda x: f"{x:4.3f}" if not pd.isna(x) else "No Run")
)
df_times["HPO Time"] = (
	df_times["HPO Time mean"].apply(lambda x: f"{x:4.3f}" if not pd.isna(x) else "No Run")
	+ " $\\pm$ "
	+ df_times["HPO Time std"].apply(lambda x: f"{x:4.3f}" if not pd.isna(x) else "No Run")
)

# Join with the existing df_metrics (verify we have the same number of rows!)
df_metrics = df_metrics.join(df_times, how="outer")

In [86]:
# # Create a time-based dataframe with elapsed times for each metric optimization
# # Using the same filtering approach as the original df_metrics
# df_filtered = df_runs_parents.loc[df_runs_parents['standardize'] == True].copy()
# df_filtered = df_filtered.loc[df_filtered['model'].isin(model_names.keys())]
# df_filtered = df_filtered.replace({"model": model_names})
# df_filtered = df_filtered.replace({"dataset_name": dataset_names})

# # Create separate dataframes for each metric optimization with time columns
# df_ari_time = df_filtered.loc[df_filtered['hpo_metric'] == 'adjusted_rand'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'ARI_best_time', 'fit_model_return_elapsed_time': 'ARI_total_time'})

# df_ami_time = df_filtered.loc[df_filtered['hpo_metric'] == 'adjusted_mutual_info'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'AMI_best_time', 'fit_model_return_elapsed_time': 'AMI_total_time'})

# df_nmi_time = df_filtered.loc[df_filtered['hpo_metric'] == 'normalized_mutual_info'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'NMI_best_time', 'fit_model_return_elapsed_time': 'NMI_total_time'})

# df_calinski_time = df_filtered.loc[df_filtered['hpo_metric'] == 'calinski_harabasz_score'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'Calinski_best_time', 'fit_model_return_elapsed_time': 'Calinski_total_time'})

# df_silhouette_time = df_filtered.loc[df_filtered['hpo_metric'] == 'silhouette'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'Silhouette_best_time', 'fit_model_return_elapsed_time': 'Silhouette_total_time'})

# df_davies_bouldin_time = df_filtered.loc[df_filtered['hpo_metric'] == 'davies_bouldin_score'][
#     ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
# ].rename(columns={'best/elapsed_time': 'Davies-Bouldin_best_time', 'fit_model_return_elapsed_time': 'Davies-Bouldin_total_time'})

# # Remove missing values before setting index
# df_ari_time = df_ari_time.dropna(subset=["ARI_best_time", "ARI_total_time"])
# df_ami_time = df_ami_time.dropna(subset=["AMI_best_time", "AMI_total_time"])
# df_nmi_time = df_nmi_time.dropna(subset=["NMI_best_time", "NMI_total_time"])
# df_calinski_time = df_calinski_time.dropna(subset=["Calinski_best_time", "Calinski_total_time"])
# df_silhouette_time = df_silhouette_time.dropna(subset=["Silhouette_best_time", "Silhouette_total_time"])
# df_davies_bouldin_time = df_davies_bouldin_time.dropna(subset=["Davies-Bouldin_best_time", "Davies-Bouldin_total_time"])

# # Set multi-index for all dataframes
# df_ari_time = df_ari_time.set_index(["dataset_name", "model"])
# df_ami_time = df_ami_time.set_index(["dataset_name", "model"])
# df_nmi_time = df_nmi_time.set_index(["dataset_name", "model"])
# df_calinski_time = df_calinski_time.set_index(["dataset_name", "model"])
# df_silhouette_time = df_silhouette_time.set_index(["dataset_name", "model"])
# df_davies_bouldin_time = df_davies_bouldin_time.set_index(["dataset_name", "model"])

# # Combine all time metrics into a single dataframe using outer join
# df_time_metrics = df_ari_time.join(df_ami_time, how="outer").join(df_nmi, how="outer").join(df_calinski_time, how="outer").join(df_silhouette_time, how="outer").join(df_davies_bouldin_time, how="outer")

# # Rename index levels
# df_time_metrics.index.names = ["Dataset", "Model"]

In [87]:
df_metrics

ARI mean   ARI std  AMI mean  \
Dataset           Model                                             
alizadeh-2000-v2  CoHiRF-SC-SRGF     0.632841  0.042223  0.757751   
                  CoHiRF-SC-SRGF-1R  0.947128  0.000000  0.922018   
                  CoHiRF-SC-SRGF-2R  0.947128  0.000000  0.922018   
                  SC-SRGF            0.891422  0.124562  0.854421   
alizadeh-2000-v3  CoHiRF-SC-SRGF     0.442624  0.012813  0.627770   
                  CoHiRF-SC-SRGF-1R  0.519160  0.000000  0.723841   
                  CoHiRF-SC-SRGF-2R  0.519160  0.000000  0.723841   
                  SC-SRGF            0.519160  0.000000  0.702416   
armstrong-2002-v1 CoHiRF-SC-SRGF     0.371617  0.039012  0.444932   
                  CoHiRF-SC-SRGF-1R  0.462629  0.116528  0.402909   
                  CoHiRF-SC-SRGF-2R  0.445998  0.110654  0.419080   
                  SC-SRGF            0.313416  0.045964  0.420515   
bittner-2000      CoHiRF-SC-SRGF     0.057890  0.022848  0.103082   
                  CoHiRF-SC-SRGF-1R  0.169038  0.019309  0.223727   
                  CoHiRF-SC-SRGF-2R  0.140731  0.010381  0.190714   
                  SC-SRGF            0.131626  0.034718  0.204931   
bredel-2005       CoHiRF-SC-SRGF     0.326251  0.024664  0.413455   
                  CoHiRF-SC-SRGF-1R  0.655139  0.036821  0.524535   
                  CoHiRF-SC-SRGF-2R  0.653397  0.035275  0.530495   
                  SC-SRGF            0.639952  0.034435  0.538158   
chowdary-2006     CoHiRF-SC-SRGF     0.702278  0.041618  0.646869   
                  CoHiRF-SC-SRGF-1R  0.923766  0.000000  0.858643   
                  CoHiRF-SC-SRGF-2R  0.923766  0.000000  0.858643   
                  SC-SRGF            0.923766  0.000000  0.858643   
ecoli             CoHiRF-SC-SRGF     0.723492  0.011082  0.667921   
                  CoHiRF-SC-SRGF-1R  0.723821  0.002943  0.681983   
                  CoHiRF-SC-SRGF-2R  0.729594  0.010561  0.695410   
                  SC-SRGF            0.713038  0.007713  0.676466   
garber-2001       CoHiRF-SC-SRGF     0.437160  0.054210  0.323151   
                  CoHiRF-SC-SRGF-1R  0.235458  0.012791  0.264929   
                  CoHiRF-SC-SRGF-2R  0.454286  0.033884  0.332909   
                  SC-SRGF            0.204396  0.035002  0.251175   
golub-1999-v2     CoHiRF-SC-SRGF     0.352280  0.043072  0.451229   
                  CoHiRF-SC-SRGF-1R  0.377647  0.053731  0.466433   
                  CoHiRF-SC-SRGF-2R  0.367010  0.054796  0.452678   
                  SC-SRGF            0.298271  0.030361  0.454763   
iris              CoHiRF-SC-SRGF     0.726146  0.031791  0.717521   
                  CoHiRF-SC-SRGF-1R  0.885958  0.008285  0.860005   
                  CoHiRF-SC-SRGF-2R  0.826558  0.036383  0.803129   
                  SC-SRGF            0.776839  0.077965  0.849873   
khan-2001         CoHiRF-SC-SRGF     0.403977  0.017035  0.552097   
                  CoHiRF-SC-SRGF-1R  0.405887  0.000000  0.555187   
                  CoHiRF-SC-SRGF-2R  0.423957  0.006823  0.573815   
                  SC-SRGF            0.364275  0.038585  0.522855   

                                          AMI std  Calinski mean  \
Dataset           Model                                            
alizadeh-2000-v2  CoHiRF-SC-SRGF     3.062353e-02       7.421898   
                  CoHiRF-SC-SRGF-1R  0.000000e+00      12.338328   
                  CoHiRF-SC-SRGF-2R  0.000000e+00      12.338328   
                  SC-SRGF            9.259718e-02      12.338328   
alizadeh-2000-v3  CoHiRF-SC-SRGF     1.897289e-02       7.411181   
                  CoHiRF-SC-SRGF-1R  0.000000e+00      12.340749   
                  CoHiRF-SC-SRGF-2R  0.000000e+00      12.340749   
                  SC-SRGF            4.790779e-02      12.340749   
armstrong-2002-v1 CoHiRF-SC-SRGF     1.701540e-02       3.423828   
                  CoHiRF-SC-SRGF-1R  1.630933e-02       4.856990   
                  CoHiRF-SC-SRGF-2R  1.638856e

The following will provide the latex code for a clean table, we only need to make a little adjustement in the first line to delete the "key" and have only one header. For the longtable environment (full data) we need to add the "\*" at the end of lines we dont want to have a page break. We also should replace the entire begin{table} ... end{table} by begin{longtable} ... end{longtable} in the latex file, if you want to put caption and labels you should break the line after with '\\' (put both on the same line!)


In [88]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{4}{*}{alizadeh-2000-v2} & CoHiRF-SC-SRGF & 0.633 $\pm$ 0.042 & 0.758 $\pm$ 0.031 & \underline{7.422 $\pm$ 0.041} & \underline{0.094 $\pm$ 0.001} & \underline{1.550 $\pm$ 0.019} & 1.503 $\pm$ 1.527 & 254.264 $\pm$ 14.525 \\
 & CoHiRF-SC-SRGF-1R & \bfseries 0.947 $\pm$ 0.000 & \bfseries 0.922 $\pm$ 0.000 & \bfseries 12.338 $\pm$ 0.000 & \bfseries 0.194 $\pm$ 0.000 & 2.460 $\pm$ 0.008 & 2.728 $\pm$ 1.494 & 465.765 $\pm$ 12.556 \\
 & CoHiRF-SC-SRGF-2R & \bfseries 0.947 $\pm$ 0.000 & \bfseries 0.922 $\pm$ 0.000 & \bfseries 12.338 $\pm$ 0.000 & \bfseries 0.194 $\pm$ 0.000 & 2.786 $\pm$ 0.100 & 1.140 $\pm$ 0.894 & 464.901 $\pm$ 13.335 \\
 & SC-SRGF & \underline{0.891 $\pm$ 0.125} & \underline{0.854 $\pm$ 0.093} & \bfseries 12.338 $\pm$ 0.000 & \bfseries 0.194 $\pm$ 0.000 & \bfseries 0.936 $\pm$ 0.014 & 0.132 $\pm$ 

# KMeans

In [56]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
datasets_to_keep = [
    "garber-2001",
    "alizadeh-2000-v2",
    "golub-1999-v2",
    "armstrong-2002-v1",
    "nursery",
    "segment",
]
models_to_keep = [
    "K-Means",
    "CoHiRF",
	"CoHiRF-1000",
    "Batch CoHiRF",
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{alizadeh-2000-v2} & Batch CoHiRF & 0.360 $\pm$ 0.014 & 0.507 $\pm$ 0.021 & 4.274 $\pm$ 0.238 & 0.049 $\pm$ 0.010 & \bfseries 0.411 $\pm$ 0.084 & 0.143 $\pm$ 0.069 & 154.717 $\pm$ 4.251 \\
 & CoHiRF & \bfseries 0.866 $\pm$ 0.007 & \bfseries 0.771 $\pm$ 0.030 & \underline{15.094 $\pm$ 0.126} & \underline{0.196 $\pm$ 0.015} & \underline{0.896 $\pm$ 0.083} & 0.054 $\pm$ 0.033 & 142.623 $\pm$ 4.180 \\
 & K-Means & \underline{0.838 $\pm$ 0.014} & \underline{0.767 $\pm$ 0.037} & \bfseries 15.151 $\pm$ 0.000 & \bfseries 0.204 $\pm$ 0.036 & 0.934 $\pm$ 0.021 & 0.012 $\pm$ 0.003 & 137.865 $\pm$ 3.673 \\
\cline{1-9}
\multirow[c]{3}{*}{armstrong-2002-v1} & Batch CoHiRF & 0.225 $\pm$ 0.071 & 0.267 $\pm$ 0.061 & 2.933 $\pm$ 0.333 & 0.003 $\pm$ 0.007 & \bfseries 0.401 $\pm$ 0.025 & 0.125 $\pm$ 0.039 & 90.402 $\pm$ 3.

# Kernel KMeans

In [57]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
datasets_to_keep = [
    "khan-2001",
    "bittner-2000",
    "iris",
    "satimage",
]
models_to_keep = [
    "Kernel RBF K-Means",
    "CoHiRF-KernelRBF",
    "Batch CoHiRF-KernelRBF",
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{bittner-2000} & Batch CoHiRF-KernelRBF & 0.088 $\pm$ 0.015 & 0.098 $\pm$ 0.011 & \bfseries 1.918 $\pm$ 0.217 & 0.041 $\pm$ 0.020 & \bfseries 0.499 $\pm$ 0.083 & 0.224 $\pm$ 0.180 & 171.649 $\pm$ 4.442 \\
 & CoHiRF-KernelRBF & \bfseries 0.104 $\pm$ 0.049 & \underline{0.136 $\pm$ 0.043} & \underline{1.569 $\pm$ 0.146} & \bfseries 0.054 $\pm$ 0.066 & \underline{0.530 $\pm$ 0.053} & 0.191 $\pm$ 0.118 & 158.762 $\pm$ 4.244 \\
 & Kernel RBF K-Means & \underline{0.090 $\pm$ 0.038} & \bfseries 0.137 $\pm$ 0.034 & 1.334 $\pm$ 0.128 & \underline{0.043 $\pm$ 0.021} & 0.905 $\pm$ 0.014 & 0.062 $\pm$ 0.017 & 148.491 $\pm$ 2.636 \\
\cline{1-9}
\multirow[c]{3}{*}{iris} & Batch CoHiRF-KernelRBF & \underline{0.687 $\pm$ 0.090} & 0.704 $\pm$ 0.073 & 121.522 $\pm$ 26.880 & 0.396 $\pm$ 0.162 & 0.889 $\pm$ 0.169 & 0.351 $\

# DBSCAN

In [58]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
datasets_to_keep = ["ecoli", "binary-alpha-digits", "segment", "chowdary-2006", "shuttle"]
models_to_keep = [
    "DBSCAN",
    "CoHiRF-DBSCAN",
    "Batch CoHiRF-DBSCAN",
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{binary-alpha-digits} & Batch CoHiRF-DBSCAN & \underline{0.006 $\pm$ 0.008} & \underline{0.032 $\pm$ 0.033} & \underline{1.473 $\pm$ 0.131} & \bfseries 0.080 $\pm$ 0.025 & \bfseries 0.837 $\pm$ 0.037 & 0.284 $\pm$ 0.238 & 58.526 $\pm$ 3.098 \\
 & CoHiRF-DBSCAN & \bfseries 0.009 $\pm$ 0.005 & \bfseries 0.035 $\pm$ 0.026 & 1.353 $\pm$ 0.109 & \underline{0.011 $\pm$ 0.035} & \underline{0.862 $\pm$ 0.035} & 0.189 $\pm$ 0.041 & 57.093 $\pm$ 1.612 \\
 & DBSCAN & 0.000 $\pm$ 0.000 & 0.002 $\pm$ 0.003 & \bfseries 1.574 $\pm$ 3.525 & -0.561 $\pm$ 0.602 & 600.462 $\pm$ 547.090 & 0.053 $\pm$ 0.036 & 44.413 $\pm$ 2.076 \\
\cline{1-9}
\multirow[c]{3}{*}{chowdary-2006} & Batch CoHiRF-DBSCAN & \underline{0.068 $\pm$ 0.053} & \underline{0.099 $\pm$ 0.047} & \underline{37.353 $\pm$ 0.000} & 0.398 $\pm$ 0.366 & \underlin

# SC-SRGF


In [59]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
datasets_to_keep = ["alizadeh-2000-v3", "alizadeh-2000-v2", "har", "satimage", "chowdary-2006"]
models_to_keep = [
    "SC-SRGF",
	"CoHiRF-SC-SRGF",
    "Batch CoHiRF-SC-SRGF",
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{2}{*}{alizadeh-2000-v2} & CoHiRF-SC-SRGF & \underline{0.633 $\pm$ 0.042} & \underline{0.758 $\pm$ 0.031} & \underline{7.422 $\pm$ 0.041} & \underline{0.094 $\pm$ 0.001} & \underline{1.550 $\pm$ 0.019} & 1.503 $\pm$ 1.527 & 254.264 $\pm$ 14.525 \\
 & SC-SRGF & \bfseries 0.891 $\pm$ 0.125 & \bfseries 0.854 $\pm$ 0.093 & \bfseries 12.338 $\pm$ 0.000 & \bfseries 0.194 $\pm$ 0.000 & \bfseries 0.936 $\pm$ 0.014 & 0.132 $\pm$ 0.061 & 142.764 $\pm$ 4.481 \\
\cline{1-9}
\multirow[c]{2}{*}{alizadeh-2000-v3} & CoHiRF-SC-SRGF & \underline{0.443 $\pm$ 0.013} & \underline{0.628 $\pm$ 0.019} & \underline{7.411 $\pm$ 0.060} & \underline{0.093 $\pm$ 0.002} & \underline{1.562 $\pm$ 0.047} & 0.928 $\pm$ 1.146 & 245.332 $\pm$ 13.850 \\
 & SC-SRGF & \bfseries 0.519 $\pm$ 0.000 & \bfseries 0.702 $\pm$ 0.048 & \bfseries 12.341 $\p

# COIL 20

In [61]:
df_latex = df_metrics.copy()
columns_to_hide = [col for col in df_latex.columns if col not in (hpo_metrics_rename + ["Best Time", "HPO Time"])]
columns_to_hide += ["NMI"]
datasets_to_keep = ["coil-20", "mnist"]
models_to_keep = [
    "K-Means",
    "CoHiRF",
	"CoHiRF-1000",
    "Batch CoHiRF",
    "Kernel RBF K-Means",
    "CoHiRF-KernelRBF",
    "Batch CoHiRF-KernelRBF",
    "DBSCAN",
    "CoHiRF-DBSCAN",
    "Batch CoHiRF-DBSCAN",
    "SC-SRGF",
    "Batch CoHiRF-SC-SRGF",
]
df_latex = df_latex.loc[
    df_latex.index.get_level_values("Dataset").isin(datasets_to_keep)
    & df_latex.index.get_level_values("Model").isin(models_to_keep),
    :,
]
highlight_max_ari = partial(highlight_max, column_name="ARI mean")
highlight_max_ami = partial(highlight_max, column_name="AMI mean")
highlight_max_calinski = partial(highlight_max, column_name="Calinski mean")
highlight_max_silhouette = partial(highlight_max, column_name="Silhouette mean")
highlight_min_davies_bouldin = partial(highlight_min, column_name="Davies-Bouldin mean")
highlight_min_best_time = partial(highlight_min, column_name="Best Time mean")
highlight_min_hpo_time = partial(highlight_min, column_name="HPO Time mean")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI mean")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI mean")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski mean")
underline_2nd_max_silhouette = partial(underline_2nd_max, column_name="Silhouette mean")
underline_2nd_min_davies_bouldin = partial(underline_2nd_min, column_name="Davies-Bouldin mean")
underline_2nd_min_best_time = partial(underline_2nd_min, column_name="Best Time mean")
underline_2nd_min_hpo_time = partial(underline_2nd_min, column_name="HPO Time mean")
print(
    df_latex.style.apply(highlight_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(underline_2nd_max_ari, subset=["ARI", "ARI mean"], axis=None)
    .apply(highlight_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(underline_2nd_max_ami, subset=["AMI", "AMI mean"], axis=None)
    .apply(highlight_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(underline_2nd_max_calinski, subset=["Calinski", "Calinski mean"], axis=None)
    .apply(highlight_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(underline_2nd_max_silhouette, subset=["Silhouette", "Silhouette mean"], axis=None)
    .apply(highlight_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .apply(underline_2nd_min_davies_bouldin, subset=["Davies-Bouldin", "Davies-Bouldin mean"], axis=None)
    .hide(columns_to_hide, axis=1)
    .to_latex(
        hrules=True,
        clines="skip-last;data",
        convert_css=True,
        column_format="ll" + "l" * (len(df_latex.columns) - len(columns_to_hide)),
        # environment="longtable",
    )
)

\begin{tabular}{lllllllll}
\toprule
 &  & ARI & AMI & Calinski & Silhouette & Davies-Bouldin & Best Time & HPO Time \\
Dataset & Model &  &  &  &  &  &  &  \\
\midrule
\multirow[c]{11}{*}{coil-20} & Batch CoHiRF & 0.381 $\pm$ 0.008 & 0.632 $\pm$ 0.010 & 62.639 $\pm$ 4.789 & 0.107 $\pm$ 0.018 & 1.532 $\pm$ 0.065 & 0.541 $\pm$ 0.153 & 172.626 $\pm$ 7.695 \\
 & Batch CoHiRF-DBSCAN & 0.335 $\pm$ 0.055 & 0.588 $\pm$ 0.051 & \bfseries 291.162 $\pm$ 71.197 & -0.001 $\pm$ 0.008 & \bfseries 0.077 $\pm$ 0.041 & 1.305 $\pm$ 1.005 & 175.064 $\pm$ 19.429 \\
 & Batch CoHiRF-KernelRBF & 0.004 $\pm$ 0.006 & 0.023 $\pm$ 0.020 & 1.804 $\pm$ 0.276 & 0.005 $\pm$ 0.002 & 2.404 $\pm$ 0.100 & 10.855 $\pm$ 17.100 & 510.956 $\pm$ 280.500 \\
 & CoHiRF & 0.355 $\pm$ 0.041 & 0.627 $\pm$ 0.017 & 287.989 $\pm$ 0.164 & \underline{0.220 $\pm$ 0.095} & 1.597 $\pm$ 0.166 & 0.290 $\pm$ 0.087 & 151.293 $\pm$ 4.393 \\
 & CoHiRF-1000 & 0.338 $\pm$ 0.009 & 0.632 $\pm$ 0.006 & 288.053 $\pm$ 0.078 & 0.180 $\pm$ 0.004 & 1.725 

# Debug and explore

In [25]:
df = df_runs_raw_2.copy()

In [26]:
df = df.loc[df["status"].isin(["FAILED", "RUNNING"])]
df

key,status,start_time,end_time,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/base_model_kwargs/n_similarities,best/base_model_kwargs/sampling_ratio,best/base_model_kwargs/sc_n_clusters,best/child_run_id,...,best/rand_score,best/silhouette,best/v_measure,fit_model_return_elapsed_time,max_memory_used,max_memory_used_after_fit,EXCEPTION,Last step finished,mlflow.parentRunId,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0002f1471a6f4a639bcfd8a3d7c90f1f,FAILED,2025-07-16 14:26:07.607,1.752676e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,21.065334,1492.632,1492.632,Expected a 2-dimensional container but got <cl...,NaN,4b3d5799ec4448719f0a0c5c291eda0c,True
0002f564f6e3405d859abc0abd6c448e,FAILED,2025-07-15 15:39:34.468,1.752594e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.013465,455.484,455.484,could not convert string to float: 'OG',NaN,742cb3f363ef47a79da84cc6e4086844,True
000303ea6e564058a7def8cddeeda1b0,FAILED,2025-07-15 13:18:50.524,1.752586e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.025954,595.816,595.816,"could not convert string to float: '""SCC""'",NaN,67862e364b9e4bf9a3a723cf9fae0873,True
00065738f4ba45a9b05cf8d62618e24b,FAILED,2025-07-15 12:30:22.107,1.752583e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.029382,591.684,591.684,"could not convert string to float: '""SCC""'",NaN,53b4533a3e62427d90f539c1e9ac0337,True
0012a8230edf41cca17eb741cbc86867,FAILED,2025-07-16 14:24:32.448,1.752676e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.015206,493.316,493.316,could not convert string to float: 'DLBCL2',NaN,181c0696d3b643758607c7dec14dfe0c,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffc589efa744f54b3378fcda6ccbcfe,FAILED,2025-11-06 11:50:01.603,1.762430e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,359.500,NaN,kth(=3) out of bounds (3),_before_fit_model,bff12221a1684a2f88776ab5859934c6,True
fffca7628fd3470d98a993a367c8aa3d,RUNNING,2025-08-12 18:54:10.939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236c3ffdfb1b441da0c1db9ca53994b3,NaN
fffe9e46b22949c48aa2eaffc64b7028,FAILED,2025-07-16 18:04:27.271,1.752689e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.067094,485.088,485.088,"could not convert string to float: '""ML2""'",NaN,8c5ee25a49bc4dd4a8c0f5960653aba4,True


In [28]:
runs_to_delete_parents = list(df.index)
df = df_runs_raw_2.copy()
df = df.loc[df["mlflow.parentRunId"].isin(runs_to_delete_parents)]
runs_to_delete_children = list(df.index)
runs_to_delete = runs_to_delete_children + runs_to_delete_parents
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ", ".join(run_uuid_query)

In [29]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

In [30]:
query = f"""
DELETE
FROM
	experiment_tags
WHERE
	experiment_id = ANY(
	SELECT
		experiment_id
	FROM
		experiments
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	latest_metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	params
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	tags
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE 
FROM 
	runs
WHERE 
	lifecycle_stage = 'deleted';

DELETE 
FROM 
	experiments
WHERE 
	lifecycle_stage = 'deleted';
"""
with engine.begin() as conn:
    conn.execute(text(query))